In [ ]:
 rm(list=ls()) 

In [ ]:
library(scRecover)
library(BiocParallel)
param <- MulticoreParam(workers = 4, progressbar = TRUE)
register(param)

In [ ]:
anno_files=c("GSM3587996_BM1.anno.txt","GSM3587997_BM2.anno.txt","GSM3587999_BM3.anno.txt", "GSM3588001_BM4.anno.txt" )
anno=list()
i=j=1
for (afile in anno_files){ 
  anno[[i]]=read.csv2(paste("BiologicalAnalysis/healthy_samples/", afile, sep=""), sep="\t", header=TRUE, row.names=1)
  i=i+1
}
exp_files=c("GSM3587996_BM1.dem.txt", "GSM3587997_BM2.dem.txt", "GSM3587998_BM3.dem.txt", "GSM3588000_BM4.dem.txt")
exp=list()
for (efile in exp_files){ 
  exp[[j]]=t(read.csv2(paste("BiologicalAnalysis/healthy_samples/", efile, sep=""), sep="\t", header=TRUE, row.names=1))
  j=j+1
}

In [ ]:
sample_label=list()
for (i in 1:length(anno)){ 
  sample_label[[i]]=cbind(rownames(exp[[i]]), anno[[i]][rownames(exp[[i]]),"CellType"])
}

In [ ]:
all_raw_exp=do.call(rbind, exp) 

In [ ]:
sample_labels=do.call(rbind, sample_label)
colnames(sample_labels)=c("CellID", "CellType")

In [ ]:
prob_cells=sample_labels[which(sample_labels[,"CellType"]=="ProB"),"CellID"]
mono_cells=sample_labels[which(sample_labels[,"CellType"]=="Mono"),"CellID"]   
sample_labels_rel=rbind(sample_labels[which(sample_labels[,"CellType"]=="ProB"),], 
                  sample_labels[which(sample_labels[,"CellType"]=="Mono"),])


In [ ]:
colnames(sample_labels_rel)=c("CellID", "CellType")
write.table(sample_labels_rel, file="cell_types_prob_mono.txt",quote=FALSE,  row.names=FALSE)

In [ ]:
raw_exp=rbind(all_raw_exp[prob_cells, ],all_raw_exp[mono_cells, ])

In [ ]:
param <- MulticoreParam(workers = 18, progressbar = TRUE)
register(param)
labels=unlist(sample_labels_rel[,2])
screcover_scImpute_exp=scRecover(counts=t(raw_exp),labels=labels, outputDir = "./Data_prob_mono/",  parallel = TRUE, BPPARAM = param)
